In [3]:
from model import Seq2Seq
from utils import load_data
import numpy as np

file_path = 'data/cmn.txt'
n_units = 256
batch_size = 64
epochs = 200
num_samples = 10000

# load data
input_texts, target_texts, input_dict, target_dict, target_dict_reverse, output_length, \
input_feature_length, output_feature_length, encoder_input, decoder_input, decoder_output = load_data(file_path, num_samples)


In [2]:
# create model
seq2seq = Seq2Seq(input_feature_length, output_feature_length, n_units)
model_train, encoder_infer, decoder_infer = seq2seq.create_model()
model_train.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print(model_train.summary())
print(encoder_infer.summary())
print(decoder_infer.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 73)]   0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None, 2623)  0           []                               
                                ]                                                                 
                                                                                                  
 lstm (LSTM)                    [(None, 256),        337920      ['input_1[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                                                 

In [3]:
# train model
model_train.fit([encoder_input, decoder_input], decoder_output, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Epoch 1/200
125/125 [==============================] - 19s 100ms/step - loss: 2.1906 - accuracy: 0.0625 - val_loss: 2.6239 - val_accuracy: 0.0620
Epoch 2/200
125/125 [==============================] - 8s 62ms/step - loss: 2.0614 - accuracy: 0.0639 - val_loss: 2.6278 - val_accuracy: 0.0618
Epoch 3/200
125/125 [==============================] - 8s 62ms/step - loss: 2.0483 - accuracy: 0.0639 - val_loss: 2.6009 - val_accuracy: 0.0626
Epoch 4/200
125/125 [==============================] - 8s 62ms/step - loss: 2.0401 - accuracy: 0.0645 - val_loss: 2.5920 - val_accuracy: 0.0669
Epoch 5/200
125/125 [==============================] - 8s 61ms/step - loss: 2.0349 - accuracy: 0.0667 - val_loss: 2.5863 - val_accuracy: 0.0678
Epoch 6/200
125/125 [==============================] - 8s 61ms/step - loss: 2.0288 - accuracy: 0.0688 - val_loss: 2.5819 - val_accuracy: 0.0693
Epoch 7/200
125/125 [==============================] - 8s 60ms/step - loss: 2.0250 - accuracy: 0.0694 - val_loss: 2.5728 - val_accurac

In [4]:
model_train.save('./result/model.h5')
encoder_infer.save('./result/encoder.h5')
decoder_infer.save('./result/decoder.h5')

In [15]:
# predict
from predict import predict_chinese
from keras.models import load_model

encoder_infer=load_model('result/encoder.h5')
decoder_infer=load_model('result/decoder.h5')

def translate(seq: str):
    input_length = max([len(i) for i in input_texts])
    encoder_input = np.zeros((1, input_length, input_feature_length))
    for char_index, char in enumerate(seq):
        encoder_input[0, char_index, input_dict[char]] = 1
    out = predict_chinese(encoder_input, encoder_infer, decoder_infer, output_length, output_feature_length)
    return out

print(translate('Hello!'))
print(translate('Nice to meet you!'))


1/1 [==============================] - 0s 23ms/step
你好。

1/1 [==============================] - 0s 24ms/step
太快了的。

